In [2]:
import requests
import threading
import json
import pandas as pd
import numpy as np
import re
import bs4
import traceback


MARGIN_COL = 5
VOTE_RATE = 0.3
URL_CHAIN = 'http://35.188.227.39:8080/enhancer/chain/scorpiosvchain'
LABEL_CHAIN = 'http://fise.iks-project.eu/ontology/entity-label'
TYPE_CHAIN = 'http://fise.iks-project.eu/ontology/entity-type'

In [16]:
def strip(s):
    return ''.join(re.split('[^a-zA-Z0-9]', s.lower()))

def get_start_of_word(s):
    it = re.finditer('[a-zA-Z0-9,\./\+]+', s)
    return [ i.start() for i in it ]

def split_row(row, pos):
    r = []
    for i in range(0, len(pos) - 1):
        r.append(row[pos[i]:pos[i + 1]])
    r.append(row[pos[-1]:])
    return r

def is_null_row(row):
    return row.isnull().sum() == len(row)

In [3]:
def request_word(word):
    try:
        r = requests.post(URL_CHAIN, data=str(word).encode('utf-8'), headers={'Content-Type': 'application/pdf'})
        r = r.json()

        res = []
        track = []
        for obj in r:
            if LABEL_CHAIN in obj:
                v = obj[LABEL_CHAIN][0]['@value']
                if strip(v) not in track:
                    res.append(v)
                    track.append(strip(v))
        return res
    except Exception as e:
        traceback.print_exc()
    
    
def request_header(word, res):
    r = request_word(word)
    if len(r) > 0:
        print(r[0])
        res.append(r[0])

def request_row(table, index, res):
    r = []
    row = table.iloc[index].dropna().tolist()
    length, width = table.shape
    print(len(row), width)
    if len(row) / width <= 0.4 and width <= 50:
        res[index] = []
        return

    threads = [ threading.Thread(target=request_header, args=(word, r)) for word in row ]

    for thread in threads:
        thread.start()

    for thread in threads:
        thread.join()

    res[index] = r

In [4]:
def separate_tables(raw_tables, table_margin=2):
    count = 0
    tables = {}
    for table in raw_tables:
        height, width = table.shape

        irow = 0
        istart = 0
        iend = 0

        while irow < height:
            while irow < height and is_null_row(table.iloc[irow]):
                irow += 1
            istart = irow

            while irow < height:
                while irow < height and (not is_null_row(table.iloc[irow])):
                    irow += 1
                iend = irow

                while irow < height and is_null_row(table.iloc[irow]):
                    irow += 1

                if irow - iend < table_margin:
                    irow += 1
                else: 
                    iend = irow 
                    break

            if iend - istart >= table_margin:
                tables['table_' + str(count)] = table.iloc[istart:irow].copy()
                count += 1

    return tables

# HTML

In [12]:
def format_html_table(raw_tables):
    for table in raw_tables:
        height, width = table.shape
        for col in table.columns:
            new_col = table[col].apply(lambda x: None if isinstance(x, str) \
                and len(x) > 0 \
                and (ord(x[0]) < 32 or ord(x[0]) > 126)  \
                else x)
            table[col] = new_col

        for ih in range(height):
            current_value = table.iloc[ih, 0]
            for iw in range(1, width):
                if table.iloc[ih, iw] == current_value:
                    table.iloc[ih, iw] = np.nan
                else:
                    current_value = table.iloc[ih, iw]
                    
                    
def separate_tables(raw_tables, table_margin=2):
    count = 0
    tables = {}
    for table in raw_tables:
        height, width = table.shape

        irow = 0
        istart = 0
        iend = 0

        while irow < height:
            while irow < height and is_null_row(table.iloc[irow]):
                irow += 1
            istart = irow

            while irow < height:
                while irow < height and (not is_null_row(table.iloc[irow])):
                    irow += 1
                iend = irow

                while irow < height and is_null_row(table.iloc[irow]):
                    irow += 1

                if irow - iend < table_margin:
                    irow += 1
                else: 
                    iend = irow 
                    break

            if iend - istart >= table_margin:
                tables['table_' + str(count)] = table.iloc[istart:irow].copy()
                count += 1

    return tables


In [26]:
with open('../data/p5materials/html/c28.html') as f:
    soup = bs4.BeautifulSoup(f, features='lxml')
    for t in soup(['script', 'style', 'meta']):
        t.extract()
    pretty_soup_str = soup.pretty()
    pretty_soup_str = re.sub('\s+<span', '<span', pretty_soup_str)
    
    
raw_tables = pd.read_html(pretty_soup_str)
format_html_table(raw_tables)
tables = separate_tables(raw_tables, table_margin=1)

tables['table_1']

TypeError: 'NoneType' object is not callable

In [29]:
arr = soup.text.replace('\xa0', ' ').split('\n')
arr

pretty_soup = 

['SSY_CLEAN_LR1_LIST_BASIS_SIKKA(07TH_NOV_20TH_DEC)',
 "SIMPSON SPENCE YOUNG(SINGAPORE)================================= 07TH NOVEMBER – 20TH DECEMBER CLEAN LR1 LIST BASIS SIKKA--------------------------- VESSEL              SIKKA DWT CBM LOA YR PORT           OPEN     FLEET        LAST CGOS                  COMMENT              ------------------- ----- --- --- --- -- -------------- -------- ------------ -------------------------- ---------------------BW RHINE            08/11 76  80  229 08 KARACHI        07/11    MENA         UMS/GO/UMS/UMS             SUB SLDENERGY CENTAUR      09/11 74  80  228 08 ANYER          31/10    SHELL        NAP/UMS/NAP/UMS            SUB SLD FUJ11NOVFOTINI LADY         10/11 71  81  228 04 FUJAIRAH       07/11    ARGYLL       NAP/GO/UMS/JET             SUBWALLEYE             10/11 70  82  228 03 FUJAIRAH       07/11    PRIMEMARINE  NAP/GO+JET/UMS/NAP         SUBENERGY CENTURION    10/11 74  78  228 08 FUJAIRAH       07/11    SHELL        UMS/NAP/NAP/NAP 

In [ ]:
OVERLAP_URL = 'http://35.186.166.22:8082/portal/servlet/service/Poheader.poi'

def overlap(word_pos, header_item_pos):
    try:
        data = json.dumps({
            'user_name': 'carrotrule_xyz.com',
            'project_name': 'DataExtractionPO-2',
            'Rule_Engine': 'RuleData-2',
            'RawJson': {
                'wX1': word_pos[0],
                'wX2': word_pos[1],
                'hX1': header_item_pos[0],
                'hX2': header_item_pos[1],
            }
        })

        r = requests.post(OVERLAP_URL, data=data)
        print(json.dumps(data, indent=2))
        print('hello', r.json())
        return None
        return r.json()['hw1o'] == 'OVERLAP'
    except Exception as e:
        traceback.print_exc()
        return word_pos[1] >= header_item_pos[0] and word_pos[0] <= header_item_pos[1]
    
r = overlap((4, 6), (7, 8))
print(r)

In [19]:
request_word('SPORE/MSIA/INDO/THAI')

[]